In [1]:
import re
import pandas as pd
import gdown as gd

## Parsing the dataa

In [2]:
url = "https://docs.google.com/document/d/1PpslvvmMsybqaGL4hVXW1PJjaRDk9hqbFm2-2NctA3c/edit?tab=t.0"
id = "1PpslvvmMsybqaGL4hVXW1PJjaRDk9hqbFm2-2NctA3c"

In [ ]:
def load_data(id):
    gd.download(id=id, output="TouristAI.txt", quiet=False)
    with open("TouristAI.txt", "r", encoding='utf-8') as f:
        content = f.read()
    return content

content = load_data(id)

Downloading...
From (original): https://drive.google.com/uc?id=1PpslvvmMsybqaGL4hVXW1PJjaRDk9hqbFm2-2NctA3c
From (redirected): https://docs.google.com/document/d/1PpslvvmMsybqaGL4hVXW1PJjaRDk9hqbFm2-2NctA3c/export?format=docx
To: c:\Users\owner\OneDrive\Desktop\Files_Deep_Learning\TouristAI.txt
29.2kB [00:00, 126kB/s]


UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 57: character maps to <undefined>

In [ ]:


def parse_attractions(content):
    # Split entries based on numbered headings like "1. "
    entries = re.split(r"\n\d+\.\s+", content)[1:]  # ignore first empty split

    parsed_list = []

    for entry in entries:
        lines = entry.strip().split("\n")

        data = {
            "title": lines[0].strip(),    # first line is "Lekki Conservation Centre (LCC)"
            "name": "",
            "state": "",
            "city": "",
            "lga": "",
            "description": "",
            "entry_fees": [],
            "opening_hours": "",
            "gps_lat": None,
            "gps_lon": None
        }

        i = 1
        while i < len(lines):
            line = lines[i].strip()

            if line.startswith("Name:"):
                data["name"] = line.replace("Name:", "").strip()

            elif line.startswith("State / City / LGA:"):
                parts = line.replace("State / City / LGA:", "").split("/")
                data["state"] = parts[0].strip()
                data["city"] = parts[1].strip()
                data["lga"] = parts[2].strip()

            elif line.startswith("Description:"):
                data["description"] = line.replace("Description:", "").strip()

            elif line.startswith("Entry fees"):
                i += 1
                while i < len(lines) and not lines[i].startswith("Opening"):
                    if lines[i].strip():
                        data["entry_fees"].append(lines[i].strip())
                    i += 1
                continue  # prevent double increment

            elif line.startswith("Opening hours:"):
                data["opening_hours"] = line.replace("Opening hours:", "").strip()

            elif line.startswith("GPS coordinates:"):
                coords = re.findall(r"[-+]?\d*\.\d+|\d+", line)
                if len(coords) >= 2:
                    data["gps_lat"] = float(coords[0])
                    data["gps_lon"] = float(coords[1])

            i += 1

        parsed_list.append(data)

    return parsed_list
